In [9]:
import pandas as pd
import sys
sys.path.append("..")
import numpy as np
from rank_bm25 import BM25Okapi
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi
from sklearn.metrics import average_precision_score
from scipy.stats import rankdata

In [10]:
valid_emb = pd.read_pickle("../pickle_backups/valid_embeddings2024-06-09T13.45.1717908334.pickle")

In [11]:
# Sample function to calculate similarity scores using GloVe or SIF embeddings
def calculate_similarity_scores(query_embedding, passage_embeddings):
    similarity_scores = []
    for passage_embedding in passage_embeddings:
        # Calculate cosine similarity between query and passage embeddings
        similarity = np.dot(query_embedding, passage_embedding) / (np.linalg.norm(query_embedding) * np.linalg.norm(passage_embedding))
        similarity_scores.append(similarity)
    return similarity_scores

# Sample function to rank texts using BM25 algorithm
def rank_texts_bm25(queries, texts):
    bm25 = BM25Okapi(texts)
    ranked_texts = [bm25.get_scores(query) for query in queries]
    return ranked_texts

def rank_texts_tfidf(queries, texts):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(texts)
    ranked_texts = [np.dot(tfidf_vectorizer.transform([query]), tfidf_matrix.T) for query in queries]
    return ranked_texts


In [12]:
queries = valid_emb['query']
texts = valid_emb['texts']

# Calculate similarity scores using GloVe embeddings
glove_queries = valid_emb['glove_queries']
glove_texts = valid_emb['glove_texts']
glove_similarity_scores = [calculate_similarity_scores(query_embedding, passage_embeddings) for query_embedding, passage_embeddings in zip(glove_queries, glove_texts)]

# Calculate similarity scores using SIF embeddings
sif_queries = valid_emb['sif_queries']
sif_texts = valid_emb['sif_texts']
sif_similarity_scores = [calculate_similarity_scores(query_embedding, passage_embeddings) for query_embedding, passage_embeddings in zip(sif_queries, sif_texts)]

/var/folders/6v/24g4rbvs2j3g23qmn8y0t_f80000gn/T/ipykernel_54267/2067005964.py:6: RuntimeWarning: invalid value encountered in divide
  similarity = np.dot(query_embedding, passage_embedding) / (np.linalg.norm(query_embedding) * np.linalg.norm(passage_embedding))
/var/folders/6v/24g4rbvs2j3g23qmn8y0t_f80000gn/T/ipykernel_54267/2067005964.py:6: RuntimeWarning: invalid value encountered in scalar divide
  similarity = np.dot(query_embedding, passage_embedding) / (np.linalg.norm(query_embedding) * np.linalg.norm(passage_embedding))


In [23]:
valid_emb['texts'][130]

{'is_selected': [0, 0, 0, 1, 0, 0, 0, 0, 0],
 'passage_text': ["Her 1963 album Etta James Rocks the House, which was cut at Nashville's New Era club, also gave her career a boost. She had other big hits in the 1960s, but mainly on the R&B charts. The song Pushover was a hit for her in 1963.",
  'Etta James. Singer, songwriter. Etta James was once among the most woefully overlooked figures in the history of blues and rock. She began finally coming into her own in the 1990s, receiving industry awards that confirmed her status as one of the matriarchs of modern music.',
  'James is the winner of four Grammys and seventeen Blues Music Awards. She was inducted into the Rock & Roll Hall of Fame in 1993, the Blues Hall of Fame in 2001, and the Grammy Hall of Fame both in 1999 and 2008. In the 1950s and 60s, she had her biggest success as a blues and R&B singer.',
  "It was that year that the young singer became Etta James (an shortened version of her first name) and her vocal group was dubbed

In [22]:
glove_similarity_scores[130]

[array([0.88580183]),
 array([0.84321393]),
 array([0.84622557]),
 array([0.88694176]),
 array([0.8716447]),
 array([0.88665299]),
 array([0.84825356]),
 array([0.89633028]),
 array([0.8546916])]

In [13]:
# Sample function to rank texts using BM25 algorithm
def rank_texts_bm25(queries, texts):
    bm25 = BM25Okapi(texts)
    ranked_texts = [bm25.get_scores(query) for query in queries]
    return ranked_texts

# Sample function to rank texts using TF-IDF algorithm
def rank_texts_tfidf(queries, texts):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(texts)
    ranked_texts = [np.dot(tfidf_vectorizer.transform([query]), tfidf_matrix.T) for query in queries]
    return ranked_texts

# Rank texts using BM25 with GloVe embeddings
glove_ranked_texts_bm25 = rank_texts_bm25(queries, texts)

# Rank texts using BM25 with SIF embeddings
sif_ranked_texts_bm25 = rank_texts_bm25(queries, texts)

# Rank texts using TF-IDF with GloVe embeddings
glove_ranked_texts_tfidf = rank_texts_tfidf(queries, texts)

# Rank texts using TF-IDF with SIF embeddings
sif_ranked_texts_tfidf = rank_texts_tfidf(queries, texts)


KeyboardInterrupt: 

In [ ]:
# Sample code for evaluation using Mean Average Precision (MAP)
def evaluate_map(ranked_texts, true_passage_indices):
    average_precisions = []
    for ranked_list, true_index in zip(ranked_texts, true_passage_indices):
        average_precision = average_precision_score(true_index, ranked_list)
        average_precisions.append(average_precision)
    mean_average_precision = np.mean(average_precisions)
    return mean_average_precision

# Sample code for evaluation using Mean Reciprocal Rank (MRR)
def evaluate_mrr(ranked_texts, true_passage_indices):
    reciprocal_ranks = []
    for ranked_list, true_index in zip(ranked_texts, true_passage_indices):
        reciprocal_rank = reciprocal_rank_score(true_index, ranked_list)
        reciprocal_ranks.append(reciprocal_rank)
    mean_reciprocal_rank = np.mean(reciprocal_ranks)
    return mean_reciprocal_rank

# Evaluate ranking performance using MAP
map_glove_bm25 = evaluate_map(glove_ranked_texts_bm25, valid_emb['true_passage_indices'])
map_sif_bm25 = evaluate_map(sif_ranked_texts_bm25, valid_emb['true_passage_indices'])
map_glove_tfidf = evaluate_map(glove_ranked_texts_tfidf, valid_emb['true_passage_indices'])
map_sif_tfidf = evaluate_map(sif_ranked_texts_tfidf, valid_emb['true_passage_indices'])

# Evaluate ranking performance using MRR
mrr_glove_bm25 = evaluate_mrr(glove_ranked_texts_bm25, valid_emb['true_passage_indices'])
mrr_sif_bm25 = evaluate_mrr(sif_ranked_texts_bm25, valid_emb['true_passage_indices'])
mrr_glove_tfidf = evaluate_mrr(glove_ranked_texts_tfidf, valid_emb['true_passage_indices'])
mrr_sif_tfidf = evaluate_mrr(sif_ranked_texts_tfidf, valid_emb['true_passage_indices'])

# Print evaluation results
print("MAP - GloVe with BM25:", map_glove_bm25)
print("MAP - SIF with BM25:", map_sif_bm25)
print("MAP - GloVe with TF-IDF:", map_glove_tfidf)
print("MAP - SIF with TF-IDF:", map_sif_tfidf)
print("MRR - GloVe with BM25:", mrr_glove_bm25)
print("MRR - SIF with BM25:", mrr_sif_bm25)
print("MRR - GloVe with TF-IDF:", mrr_glove_tfidf)
print("MRR - SIF with TF-IDF:", mrr_sif_tfidf)

In [ ]:
textstextstextstextstextstextstextstextstextstextstextstextstextstextstextstextstextstextstextstextstextstextstextstextstextstextstextstexts